In [1]:
import pandas as pd
from datetime import datetime, timedelta

#load dataset from database inaportnet
df1 = pd.read_excel("D:\Analisis Inaportnet\data PKK 2022\PKK.IDSRI.01.2022.xlsx")
df2 = pd.read_excel("D:\Analisis Inaportnet\data PKK 2022\PKK.IDSRI.02.2022.xlsx")
df3 = pd.read_excel("D:\Analisis Inaportnet\data PKK 2022\PKK.IDSRI.03.2022.xlsx")
df4 = pd.read_excel("D:\Analisis Inaportnet\data PKK 2022\PKK.IDSRI.04.2022.xlsx")
df5 = pd.read_excel("D:\Analisis Inaportnet\data PKK 2022\PKK.IDSRI.05.2022.xlsx")
df6 = pd.read_excel("D:\Analisis Inaportnet\data PKK 2022\PKK.IDSRI.06.2022.xlsx")
df7 = pd.read_excel("D:\Analisis Inaportnet\data PKK 2022\PKK.IDSRI.07.2022.xlsx")
df8 = pd.read_excel("D:\Analisis Inaportnet\data PKK 2022\PKK.IDSRI.08.2022.xlsx")

#append dataset database inaportnet
ina_database = pd.concat([df1,df2,df3,df4,df5,df6,df7,df8])

#load dataset from database inaportnet and scraping inaportnet
#ina_database = pd.read_excel("D:\databasePKK.xlsx")
ina_scrape = pd.read_excel("D:\scrapePKK.xlsx")

#join dataset inaportnet database and scraping
vlookup = pd.merge(ina_database, ina_scrape, left_on=['No PKK'], right_on=['nomor produk'], how='inner')

#change waktu permohonan into Local Time
vlookup['waktu permohonan LT'] = vlookup['waktu permohonan']+timedelta(hours=8)

#calculate time difference
vlookup['selisih waktu'] = vlookup['waktu respon'] - vlookup['waktu permohonan LT']

#make criteria in time difference
def sel(vlookup):
    if vlookup['selisih waktu'] < timedelta(seconds=31):
        return "Kurang dari 30 detik"
    elif vlookup['selisih waktu'] < timedelta(minutes=30):
        return "Antara 30 detik - 30 menit"
    elif vlookup['selisih waktu'] < timedelta(hours=1):
        return "Antara 30 - 60 menit"
    elif vlookup['selisih waktu'] < timedelta(hours=6):
        return "Antara 1 - 6 jam"
    elif vlookup['selisih waktu'] < timedelta(hours=12):
        return "Antara 6 - 12 jam"
    elif vlookup['selisih waktu'] < timedelta(days=1):
        return "Antara 12 - 24 jam"
    else:
        return "Lebih dari 24 jam"

vlookup['pembagian selisih waktu'] = vlookup.apply(sel, axis=1)
sort_data = vlookup.sort_values("selisih waktu", axis = 0, ascending = True)

#grouping pembagian selisih waktu
waktu_pelayanan = vlookup.groupby('pembagian selisih waktu')['pembagian selisih waktu'].count().reset_index(name='jumlah')
waktu_pelayanan['persentase'] = waktu_pelayanan['jumlah']/len(vlookup)*100
waktu_pelayanan.head(7)

#indentasi penomoran waktu pelayanan
def nomor_waktu(waktu_pelayanan):
    if waktu_pelayanan['pembagian selisih waktu'] == 'Kurang dari 30 detik':
        return 1
    elif waktu_pelayanan['pembagian selisih waktu'] == 'Antara 30 detik - 30 menit':
        return 2
    elif waktu_pelayanan['pembagian selisih waktu'] == 'Antara 30 - 60 menit':
        return 3
    elif waktu_pelayanan['pembagian selisih waktu'] == 'Antara 1 - 6 jam':
        return 4
    elif waktu_pelayanan['pembagian selisih waktu'] == 'Antara 6 - 12 jam':
        return 5
    elif waktu_pelayanan['pembagian selisih waktu'] == 'Antara 12 - 24 jam':
        return 6
    else:
        return 7

#grouping waktu pelayanan
waktu_pelayanan['Nomor'] = waktu_pelayanan.apply(nomor_waktu, axis=1)
waktu_pelayanan = waktu_pelayanan.sort_values("Nomor", axis = 0, ascending = True)

#membuat table hari, bulan, waktu permohonan
#divide waktu permohonan into day, month, time
month_mapping={
    1: 'January', 
    2: 'February', 
    3: 'March', 
    4: 'April', 
    5: 'May',
    6: 'June', 
    7: 'July',
    8: 'August'
} 
dw_mapping={
    0: 'Monday', 
    1: 'Tuesday', 
    2: 'Wednesday', 
    3: 'Thursday', 
    4: 'Friday',
    5: 'Saturday', 
    6: 'Sunday'
} 

vlookup['bulan permohonan'] = vlookup['waktu permohonan LT'].dt.month.map(month_mapping)
vlookup['hari permohonan'] = vlookup['waktu permohonan LT'].dt.dayofweek.map(dw_mapping)

#grouping pembagian bulan permohonan
bulan_permohonan = vlookup.groupby('bulan permohonan')['bulan permohonan'].count().reset_index(name='jumlah')
bulan_permohonan['persentase'] = bulan_permohonan['jumlah']/len(vlookup)*100

def nomor(bulan_permohonan):
    if bulan_permohonan['bulan permohonan'] == 'January':
        return 1
    elif bulan_permohonan['bulan permohonan'] == 'February':
        return 2
    elif bulan_permohonan['bulan permohonan'] == 'March':
        return 3
    elif bulan_permohonan['bulan permohonan'] == 'April':
        return 4
    elif bulan_permohonan['bulan permohonan'] == 'May':
        return 5
    elif bulan_permohonan['bulan permohonan'] == 'June':
        return 6
    elif bulan_permohonan['bulan permohonan'] == 'July':
        return 7
    else:
        return 8

bulan_permohonan['nomor'] = bulan_permohonan.apply(nomor, axis=1)
bulan_permohonan = bulan_permohonan.sort_values("nomor", axis = 0, ascending = True)
bulan_permohonan

#grouping pembagian bulan permohonan
def nomor_hari(hari_permohonan):
    if hari_permohonan['hari permohonan'] == 'Monday':
        return 1
    elif hari_permohonan['hari permohonan'] == 'Tuesday':
        return 2
    elif hari_permohonan['hari permohonan'] == 'Wednesday':
        return 3
    elif hari_permohonan['hari permohonan'] == 'Thursday':
        return 4
    elif hari_permohonan['hari permohonan'] == 'Saturday':
        return 5
    elif hari_permohonan['hari permohonan'] == 'Sunday':
        return 6
    else:
        return 7
    
hari_permohonan = vlookup.groupby('hari permohonan')['hari permohonan'].count().reset_index(name='jumlah')
hari_permohonan['Nomor'] = hari_permohonan.apply(nomor_hari, axis=1)
hari_permohonan = hari_permohonan.sort_values("Nomor", axis = 0, ascending = True)
hari_permohonan

#pembagian kriteria berdasarkan waktu permohonan
batas_waktu1 = pd.to_datetime('06:00:00', format='%H:%M:%S')
batas_waktu2 = pd.to_datetime('12:00:00', format='%H:%M:%S')
batas_waktu3 = pd.to_datetime('18:00:00', format='%H:%M:%S')

#kriteria jam permohonan
def jam(vlookup):
    if vlookup['jam permohonan'] < batas_waktu1:
        return "Jam 00.00 s.d. 06.00"
    elif vlookup['jam permohonan'] < batas_waktu2:
        return "Jam 06.00 s.d. 12.00"
    elif vlookup['jam permohonan'] < batas_waktu3:
        return "Jam 12.00 s.d. 18.00"
    else:
        return "Jam 18.00 s.d. 24.00"

vlookup['jam permohonan'] = vlookup['waktu permohonan LT'].dt.time
vlookup['jam permohonan'] = pd.to_datetime(vlookup['jam permohonan'], format='%H:%M:%S')
vlookup['pembagian jam permohonan'] = vlookup.apply(jam, axis=1)

#grouping pembagian jam permohonan
waktu_permohonan = vlookup.groupby('pembagian jam permohonan')['pembagian jam permohonan'].count().reset_index(name='jumlah')
waktu_permohonan['persentase'] = waktu_permohonan['jumlah']/len(vlookup)*100

FileNotFoundError: [Errno 2] No such file or directory: 'D:\\Analisis Inaportnet\\data PKK 2022\\PKK.IDSRI.01.2022.xlsx'

In [2]:
waktu_pelayanan

,pembagian selisih waktu,jumlah,persentase,Nomor
5,Kurang dari 30 detik,231,1.307374,1
3,Antara 30 detik - 30 menit,7588,42.945271,2
2,Antara 30 - 60 menit,2883,16.316713,3
0,Antara 1 - 6 jam,5295,29.967740,4
4,Antara 6 - 12 jam,1016,5.750184,5
1,Antara 12 - 24 jam,598,3.384459,6
6,Lebih dari 24 jam,58,0.328259,7


In [32]:
bulan_permohonan

,bulan permohonan,jumlah,persentase,nomor
3,January,616,3.486332,1
2,February,2790,15.790367,2
6,March,2605,14.743336,3
0,April,2622,14.839549,4
7,May,2441,13.815156,5
5,June,2715,15.365895,6
4,July,2702,15.292320,7
1,August,1178,6.667044,8


In [33]:
hari_permohonan

,hari permohonan,jumlah,Nomor
1,Monday,3145,1
5,Tuesday,2722,2
6,Wednesday,2804,3
4,Thursday,2886,4
2,Saturday,1891,5
3,Sunday,1341,6
0,Friday,2880,7


In [34]:
waktu_permohonan

,pembagian jam permohonan,jumlah,persentase
0,Jam 00.00 s.d. 06.00,579,3.276926
1,Jam 06.00 s.d. 12.00,6855,38.796763
2,Jam 12.00 s.d. 18.00,7095,40.155074
3,Jam 18.00 s.d. 24.00,3140,17.771238
